<a href="https://colab.research.google.com/github/tahasama/recipe-api/blob/master/AI_Django.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('thank you for this tool')

thank you for this tool


In [ ]:
import pandas as pd

# Data Science




In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
def get_title(name):
    if '.' in name:
        return name.split(',')[1].split('.')[0].strip()
    else:
        return 'Unknown'

In [ ]:
titles = sorted(set([x for x in df.Name.map(lambda x: get_title(x))]))

In [ ]:
def replace_titles(x):
    title = x['Title']
    if title in ['Capt', 'Col', 'Major']:
        return 'Officer'
    elif title in ["Jonkheer","Don",'the Countess', 'Dona', 'Lady',"Sir"]:
        return 'Royalty'
    elif title in ['the Countess', 'Mme', 'Lady']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms']:
        return 'Miss'
    else:
        return title

In [ ]:
df['Title'] = df['Name'].map(lambda x: get_title(x))
df['Title'] = df.apply(replace_titles, axis=1)

In [ ]:
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Fare'].fillna(df['Fare'].median(), inplace=True)
df['Embarked'].fillna("S", inplace=True)
df.drop("Cabin", axis=1, inplace=True)
df.drop("Ticket", axis=1, inplace=True)
df.drop("Name", axis=1, inplace=True)

In [ ]:
df.Sex.replace(('male','female'), (0,1), inplace = True)
df.Embarked.replace(('S','C','Q'), (0,1,2), inplace = True)
df.Title.replace(('Mr','Miss','Mrs','Master','Dr','Rev','Officer','Royalty'), 
                 (0,1,2,3,4,5,6,7), inplace = True)

In [ ]:
corr = df.corr()
corr.Survived.sort_values(ascending=True)

Pclass        -0.338481
Age           -0.064910
SibSp         -0.035322
PassengerId   -0.005007
Parch          0.081629
Embarked       0.106811
Fare           0.257307
Title          0.361340
Sex            0.543351
Survived       1.000000
Name: Survived, dtype: float64

# Machine Learning

In [ ]:
from sklearn.model_selection import train_test_split
import pickle
from sklearn.ensemble import RandomForestClassifier

In [ ]:
predictors = df.drop(['Survived', 'PassengerId'], axis=1)
target = df["Survived"]
x_train, x_val, y_train, y_val = train_test_split(
                                        predictors, target, test_size = 0.22, random_state = 0)

In [ ]:
randomforest = RandomForestClassifier()
randomforest.fit(x_train, y_train)
y_pred = randomforest.predict(x_val)

In [ ]:
filename = 'titanic_model.sav'
pickle.dump(randomforest, open(filename, 'wb'))

In [ ]:
from sklearn.metrics import accuracy_score
acc = round(accuracy_score(y_pred, y_val)*100, 2)
print(acc)

83.25


In [ ]:
df_test = pd.read_csv("test.csv")

df_test['Title'] = df_test['Name'].map(lambda x: get_title(x))
df_test['Title'] = df_test.apply(replace_titles, axis=1)

ids = df_test['PassengerId']

df_test['Age'].fillna(df_test['Age'].median(), inplace=True)
df_test['Fare'].fillna(df_test['Fare'].median(), inplace=True)
df_test['Embarked'].fillna("S", inplace=True)
df_test.drop("Cabin", axis=1, inplace=True)
df_test.drop("Ticket", axis=1, inplace=True)
df_test.drop("Name", axis=1, inplace=True)
df_test.drop("PassengerId", axis=1, inplace=True)
df_test.Sex.replace(('male','female'), (0,1), inplace = True)
df_test.Embarked.replace(('S','C','Q'), (0,1,2), inplace = True)
df_test.Title.replace(('Mr','Miss','Mrs','Master','Dr','Rev','Officer','Royalty'), 
                 (0,1,2,3,4,5,6,7), inplace = True)

In [ ]:
predictions = randomforest.predict(df_test)
output = pd.DataFrame({'PassengerId': ids, 'Survived': predictions})
output.to_csv('submission.csv', index=False)

In [ ]:
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>